## Prelude

Pythran is a compiler that turns numerical kernels into native modules.

You can download it on:

- PyPI: ``pip install pythran``
- Conda: ``conda install pythran``

Linux, OSX and Windows (through WinPython) are supported.

Partial Python3 support.

# Introduction with Pi computation

Computing $\pi$ is quite old fashined, but it's a good start to learn Pythran!

Here is a Fortran-like version:

In [1]:
def pi_approximate(n):
    step = 1.0 / n
    result = 0   
    for i in range(n):
        x = (i + 0.5) * step
        result += 4.0 / (1.0 + x * x)
    return step * result

pi_approximate(1000000)

3.1415926535897643

We can get a first glimpse of its performance using the ``timeit`` module:

In [2]:
%timeit pi_approximate(1000000)

170 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Turning this code into Pythran code is relatively easy. First we need to import Pythran:

In [3]:
import pythran

WARNING  Pythran support disabled for module: omp


And load it's notebook integration mode:

In [4]:
%load_ext pythran.magic
%env CC='gcc-7'
%env CXX='g++-7'

env: CC='gcc-7'
env: CXX='g++-7'


Now we'll just reproduce the code above, with an additionnal line to tell pythran about the argument type. the return type is infered.

In [5]:
%%pythran
#pythran export pi_approximate_pythran(int)
import numpy as np
def pi_approximate_pythran(n):
    step = 1.0 / n
    result = 0   
    for i in range(n):
        x = (i + 0.5) * step
        result += 4.0 / (1.0 + x * x)
    return step * result

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/__builtin__/range.hpp:6:0,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmp29c5cmth.cpp:15:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/__builtin__/xrange.hpp:102:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rend() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rend() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/__builtin__/xrange.hpp:97:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rbegin() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rbegin() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py

Hopefully, the code behaves the same:

In [6]:
pi_approximate_pythran(1000000)

3.1415926535897643

But it runs faster!

In [7]:
%timeit pi_approximate_pythran(1000000)

3.63 ms ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Can we go faster? The astute reader has already noticed that the loop can run in parallel, so let's use OpenMP integration:

In [8]:
%%pythran -fopenmp
#pythran export pi_approximate_pythran_omp(int)
import numpy as np
def pi_approximate_pythran_omp(n):
    step = 1.0 / n
    result = 0
    #omp parallel for reduction(+:result)
    for i in range(n):
        x = (i + 0.5) * step
        result += 4.0 / (1.0 + x * x)
    return step * result

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/__builtin__/range.hpp:6:0,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpozqajthp.cpp:15:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/__builtin__/xrange.hpp:102:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rend() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rend() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/__builtin__/xrange.hpp:97:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rbegin() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rbegin() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py

In [9]:
%timeit pi_approximate_pythran_omp(1000000)

1e+03 µs ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


But everything looks very Fortran-ish in this example. Why not trying the following:

In [10]:
import numpy as np
def pi_numpy_style(n):
    step = 1.0 / n
    x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
    return step * np.sum(4. / (1. + x ** 2))

It works the same, but it's already faster as most of the computations are done using native code:

In [11]:
pi_numpy_style(1000000)

3.1415926535898775

In [12]:
%timeit pi_numpy_style(1000000)

8.86 ms ± 34.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Good new! Pythran can also handle this version, without much changes:

In [13]:
%%pythran
#pythran export pi_numpy_style_pythran(int)
import numpy as np
def pi_numpy_style_pythran(n):
    step = 1.0 / n
    x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
    return step * np.sum(4. / (1. + x ** 2))

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/div.hpp:7:0,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpeer_k62w.cpp:16:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp: In function 'bool pythonic::operator_::lshift(bool, bool)':
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:48: warning: '<<' in boolean context, did you mean '<' ? [-Wint-in-bool-context]
     DEFINE_ALL_OPERATOR_OVERLOADS_IMPL(lshift, << )
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/overloads.hpp:9:14: note: in definition of macro 'PYTHONIC_OPERATOR_OVERLOAD_IMPL'
     return a op b;                                                             \
              ^~
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:5: note:

     ^~~~~~~~~~~~


It still works, and runs almost as fast as the numpy-free version converted by Pythran:

In [14]:
pi_numpy_style_pythran(1000000)

3.1415926535897643

In [15]:
%timeit pi_numpy_style_pythran(1000000)

5.68 ms ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Cherry on the cake: Pythran can take advantage of the vectorized code to generate SIMD code:

In [16]:
%%pythran -DUSE_BOOST_SIMD -march=native
#pythran export pi_numpy_style_pythran_simd(int)
import numpy as np
def pi_numpy_style_pythran_simd(n):
    step = 1.0 / n
    x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
    return step * np.sum(4. / (1. + x ** 2))

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/div.hpp:7:0,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpivl7gyzy.cpp:16:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp: In function 'bool pythonic::operator_::lshift(bool, bool)':
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:48: warning: '<<' in boolean context, did you mean '<' ? [-Wint-in-bool-context]
     DEFINE_ALL_OPERATOR_OVERLOADS_IMPL(lshift, << )
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/overloads.hpp:9:14: note: in definition of macro 'PYTHONIC_OPERATOR_OVERLOAD_IMPL'
     return a op b;                                                             \
              ^~
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:5: note:

       ^~~~~~~~~~~~~~~~~~~~~~~~


In [17]:
%timeit pi_numpy_style_pythran_simd(1000000)

3.94 ms ± 84.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Pythran in a nutshell

- DSL embeded into Python (no technological debt)
- Minimalists type annotations (only the exported functions)
- Parallelization and Vectorization are possible
- Supports (an already large part of) Numpy and Python builtins

# Type Annotations

Consider the following function:

In [18]:
def pairwise_distance(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))

It makes use of fancy indexing, broadcasting and Numpy. And it's polymorphic!

In [19]:
size = 100
args32 = np.random.random((size, size)).astype(np.float32)
args64 = np.random.random((size, size)).astype(np.float64)  #cast useless
%timeit pairwise_distance(args32)
%timeit pairwise_distance(args64)

1.2 ms ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.3 ms ± 18.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Pythran can handle all of this! Note the double export to specify the overloads:

In [20]:
%%pythran
import numpy as np
#pythran export pairwise_distance_pythran(float32[][])
#pythran export pairwise_distance_pythran(float64[:,:])
def pairwise_distance_pythran(X):
    print (X.dtype)
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/add.hpp:7:0,
                 from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/types/numpy_operators.hpp:7,
                 from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/types/ndarray.hpp:1132,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmp5qum_0w0.cpp:14:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp: In function 'bool pythonic::operator_::lshift(bool, bool)':
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:48: warning: '<<' in boolean context, did you mean '<' ? [-Wint-in-bool-context]
     DEFINE_ALL_OPERATOR_OVERLOADS_IMPL(lshift, << )
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/overloads.hpp:9:14: note: in definiti

/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/types/numpy_expr.hpp:22:27: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]


In [22]:
print(pairwise_distance_pythran(args32).dtype)
%timeit pairwise_distance_pythran(args32)
print(pairwise_distance_pythran(args64).dtype)
%timeit pairwise_distance_pythran(args64)

float32
884 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
float64
912 µs ± 2.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Pythran also automatically handles transposed arguments, without making a copy:

In [23]:
pairwise_distance_pythran(args64.T)

array([[ 3.69724678,  3.86923677,  3.88354504, ...,  3.77170682,
         4.49111697,  4.28071042],
       [ 4.046676  ,  4.22076358,  4.41779641, ...,  4.19698134,
         4.15956303,  4.1780762 ],
       [ 3.86027042,  4.04092985,  4.31161638, ...,  4.2475385 ,
         4.0949077 ,  4.29476303],
       ..., 
       [ 4.29501873,  4.44832888,  4.13206678, ...,  4.19163173,
         4.42579207,  3.6608686 ],
       [ 4.17561149,  4.12818252,  3.92522576, ...,  3.93665455,
         4.2331271 ,  4.32786092],
       [ 3.80711773,  4.35833347,  4.3049985 , ...,  4.17900989,
         4.43317329,  4.04168381]])

There's more than arrays and scalars in Pythran types. What about... a tuple of tuple of complex numbers, lists and sets?

In [24]:
%%pythran
#pythran export wtf((int, complex128, (int set, int list, int:str dict)))
def wtf(x):
    return x

In [25]:
wtf(1)

TypeError: Invalid argument type for pythranized function `wtf'.
Candidates are:
   wtf(decltype(make_tuple(std.declval<long>(), std.declval<std.complex<double>>(), std.declval<decltype(make_tuple(std.declval<set<long>>(), std.declval<list<long>>(), std.declval<dict<long,str>>()))>())))


In [26]:
strange_arg = 42, 1 + 1j, ({1, 2, 3}, [1, 2,3], {1: 'unan', 2: 'daou', 3: 'tri'})
wtf(strange_arg)

(42, (1+1j), ({1, 2, 3}, [1, 2, 3], {1: 'unan', 2: 'daou', 3: 'tri'}))

Beware that Pythran works on a copy when passing ``tuple``, ``list``, ``set`` or ``dict`` in the Pythran world (it's ok for ``ndarray`` as it does not copy the whole data):

In [27]:
wtf(strange_arg) is strange_arg

False

Sometimes, you feel like using very long function prototypes. In that case use multi-line exports:

In [28]:
%%pythran

#pythran export my(bool,
#                  bool,
#                  bool,
#                  bool)
def my(ga, bu, zo, meu):
    pass

In [29]:
my

<function pythranized_91f76c58891b91073f8e4e4dae8d0989.my>

In [30]:
type(my(True, True, True, True))

NoneType

Default arguments are taken into account, but they must be exported explictely:

In [31]:
%%pythran
#pythran export pi_numpy_style_pythran_default(int)
#pythran export pi_numpy_style_pythran_default()
import numpy as np
def pi_numpy_style_pythran_default(n=1):
    step = 1.0 / n
    x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
    return step * np.sum(4. / (1. + x ** 2))

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/div.hpp:7:0,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpavrvsnzu.cpp:16:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp: In function 'bool pythonic::operator_::lshift(bool, bool)':
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:48: warning: '<<' in boolean context, did you mean '<' ? [-Wint-in-bool-context]
     DEFINE_ALL_OPERATOR_OVERLOADS_IMPL(lshift, << )
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/overloads.hpp:9:14: note: in definition of macro 'PYTHONIC_OPERATOR_OVERLOAD_IMPL'
     return a op b;                                                             \
              ^~
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:5: note:

     ^~~~~~~~~~~~


In [32]:
pi_numpy_style_pythran_default()

3.2

In [33]:
pi_numpy_style_pythran_default(10)

3.2

# Compilation of Numpy Expressions

Pythran is well aware of high-level numpy expressions. Consider this function:

In [34]:
import numpy as np
def vibr_energy(harmonic, anharmonic, i):
    return np.exp(-harmonic * i - anharmonic * (i ** 2))

dat0, dat1 = np.random.random(1000000), np.random.random(1000000)

In [35]:
%timeit vibr_energy(dat0, dat1, 3.)

12.7 ms ± 99.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


A typical way to optimize it would be to use the ``numexpr`` package:

In [36]:
import numexpr as ne
def vibr_energy_numexpr(harmonic, anharmonic, i):
    return ne.evaluate('exp(-harmonic * i - anharmonic * (i ** 2))')

In [37]:
vibr_energy_numexpr(dat0, dat1, 3.)  # maybe ne has a cache?
%timeit vibr_energy_numexpr(dat0, dat1, 3.)

4.02 ms ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Pythran implements (roughly) the same optimizations as ``numexpr`` does:

In [38]:
%%pythran -DUSE_BOOST_SIMD -march=native -Ofast -fopenmp

import numpy as np
#pythran export vibr_energy_pythran(float[], float[], float)

def vibr_energy_pythran(harmonic, anharmonic, i):
    
    return np.exp(-harmonic * i - anharmonic * (i ** 2))

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/add.hpp:7:0,
                 from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/types/numpy_operators.hpp:7,
                 from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/types/ndarray.hpp:1132,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmp2v27jsco.cpp:12:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp: In function 'bool pythonic::operator_::lshift(bool, bool)':
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:48: warning: '<<' in boolean context, did you mean '<' ? [-Wint-in-bool-context]
     DEFINE_ALL_OPERATOR_OVERLOADS_IMPL(lshift, << )
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/overloads.hpp:9:14: note: in definiti

       ^~~~~~~~~~~~~~~~~~~~~~~~


In [39]:
%timeit vibr_energy_pythran(dat0, dat1, 3.)

2.63 ms ± 23.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Remember that Pythran can handle polymorphic code? Then let's try:

In [40]:
%%pythran
import numpy as np
#pythran export vibr_energy_pythran(float[], float[], float)
#pythran export vibr_energy_pythran(float[], float[], float[])
def vibr_energy_pythran(harmonic, anharmonic, i):
    return np.exp(-harmonic * i - anharmonic * (i ** 2))

In file included from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/add.hpp:7:0,
                 from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/types/numpy_operators.hpp:7,
                 from /usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/types/ndarray.hpp:1132,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmp7uvn99wn.cpp:12:
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp: In function 'bool pythonic::operator_::lshift(bool, bool)':
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/lshift.hpp:21:48: warning: '<<' in boolean context, did you mean '<' ? [-Wint-in-bool-context]
     DEFINE_ALL_OPERATOR_OVERLOADS_IMPL(lshift, << )
/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran/pythonic/operator_/overloads.hpp:9:14: note: in definiti

     ^~~~~~~~~~~~


In [41]:
%timeit vibr_energy_pythran(dat0, dat1, dat0)

14.4 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Broadcasting on the way!

# Using Pythran from the Command Line

Pythran can be used without a Jupyter notebook! It requires you to

1. Write your code to Pythranize into a seperate file;
2. Call the Pythran compiler.

In [42]:
%%file scrabble.py
#pythran export scrabble_score(str, str:int dict)
def scrabble_score(word, scoretable):
    score = 0
    for letter in word:
        if letter in scoretable:
            score += scoretable[letter]
    return score


Overwriting scrabble.py


Using the package API, or simply ``pythran scrabble.py``

In [43]:
!python -m pythran.run -v scrabble.py

/usr/bin/python: No module named pythran


In [44]:
import scrabble

In [45]:
scrabble.__file__

'/Users/navaro/PycharmProjects/notebooks-python/scrabble.so'

In [46]:
scrabble.scrabble_score("hello", {"h": 4, "e": 1, "l": 1, "o": 1})

8

# Using Pythran with distutils

Pythran provides some facilities for distutils integration, in the form of a ``PythranExtension``:

In [47]:
%%file setup.py
from distutils.core import Extension
from setuptools import setup, dist

dist.Distribution(dict(setup_requires='pythran'))

from pythran import PythranExtension
module1 = PythranExtension('demo', sources = ['scrabble.py'])

setup(name = 'demo',
      version = '1.0',
      description = 'This is a demo package',
      ext_modules = [module1])


Writing setup.py


In [49]:
!python3 setup.py build -v

WARNING  Pythran support disabled for module: omp
running build
running build_ext
building 'demo' extension
C compiler: gcc-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

creating build/temp.macosx-10.12-x86_64-3.6
compile options: '-DUSE_GMP -DBOOST_SIMD_CONFIG_SUPPORT_POSIX_MEMALIGN -DENABLE_PYTHON_MODULE -D__PYTHRAN__=3 -DPYTHRAN_BLAS_OPENBLAS -I/usr/local/opt/openblas/include -I/usr/local/lib/python3.6/site-packages/pythran-0.8.3-py3.6.egg/pythran -I/usr/local/lib/python3.6/site-packages/numpy/core/include -I/usr/local/include -I/usr/local/opt/openssl/include -I/usr/local/opt/sqlite/include -I/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/include/python3.6m -c'
extra options: '-std=c++11 -fno-math-errno'
gcc-7: scrabble.cpp
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
creating build/lib.macosx-10.12-x86_64-3.6
g++-7 -bundle -un

In [50]:
!python3 setup.py sdist

WARNING  Pythran support disabled for module: omp
running sdist
running egg_info
creating demo.egg-info
writing demo.egg-info/PKG-INFO
writing dependency_links to demo.egg-info/dependency_links.txt
writing top-level names to demo.egg-info/top_level.txt
writing manifest file 'demo.egg-info/SOURCES.txt'
reading manifest file 'demo.egg-info/SOURCES.txt'
writing manifest file 'demo.egg-info/SOURCES.txt'
running check


creating demo-1.0
creating demo-1.0/demo.egg-info
copying files to demo-1.0...
copying README.md -> demo-1.0
copying scrabble.cpp -> demo-1.0
copying setup.py -> demo-1.0
copying demo.egg-info/PKG-INFO -> demo-1.0/demo.egg-info
copying demo.egg-info/SOURCES.txt -> demo-1.0/demo.egg-info
copying demo.egg-info/dependency_links.txt -> demo-1.0/demo.egg-info
copying demo.egg-info/top_level.txt -> demo-1.0/demo.egg-info
Writing demo-1.0/setup.cfg
creating dist
Creating tar archive
removing 'demo-1.0' (and everything under it)


In [51]:
!tar tzf dist/demo-1.0.tar.gz

demo-1.0/
demo-1.0/demo.egg-info/
demo-1.0/demo.egg-info/dependency_links.txt
demo-1.0/demo.egg-info/PKG-INFO
demo-1.0/demo.egg-info/SOURCES.txt
demo-1.0/demo.egg-info/top_level.txt
demo-1.0/PKG-INFO
demo-1.0/README.md
demo-1.0/scrabble.cpp
demo-1.0/setup.cfg
demo-1.0/setup.py


# Getting Help

- GitHub: https://github.com/serge-sans-paille/pythran
- Mailing list: http://www.freelists.org/list/pythran
- IRC: #pythran on FreeNode
- StackOverflow: http://stackoverflow.com/questions/tagged/pythran

# Misc

Things you probably don't want to know, but they were fun to implement, so let's talk about them anyway :-)

## Functions as regular values

In [52]:
%%pythran
#pythran export modify(int, str)
actions = {"increase": lambda x: x + 1,
           "decrease": lambda x: x - 1}

def modify(value, action):
    what = actions[action]
    return what(value)

In [53]:
modify(1, "increase")

2

Passing functions in and out is not supported though.

## Big Numbers

Not widely supported, but it works for simple examples.

In [57]:
%%pythran
#pythran export factorize_naive(long)
def factorize_naive(n):
    if n < 2:
        return []
    
    factors = []
    
    p = 2

    while True:
        if n == 1:
            return factors

        r = n % p
        if r == 0:
            factors.append(p)
            n = n / p
        elif p * p >= n:
            factors.append(n)
            return factors
        elif p > 2:
            # Advance in steps of 2 over odd numbers
            p += 2
        else:
            # If p == 2, get to 3
            p += 1
    assert False, "unreachable"

NotImplementedError: <class 'type'>:<class 'pythran.tables.long'>

In [55]:
%timeit factorize_naive(3241618756762348687L)

SyntaxError: invalid syntax (<unknown>, line 1)

### Cleanup before you leave the room ;-)

In [56]:
!rm -rf build dist scrabble.py setup.py scrabble.so # cleanup